<a href="https://colab.research.google.com/github/christinevu510/Drug-Classification/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Drug Classification**

Mackenzie Carter, Vannessa Salazar, and Christine Vu

Shiley-Marcos School of Engineering, University of San Diego

ADS 509: Applied Text Mining

Professor Anna Marbut

October 23, 2023

***

## **Data Importing**

### **Import Libraries**

In [ ]:
#Import Packages
import requests
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import sklearn.metrics
import json
import sqlite3
import nltk
import random
import string
import re

from pandas.io.json import json_normalize
from collections import Counter, defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

### **Obtain URLs**

In [ ]:
# URLs
anx_URL = "https://api.fda.gov/drug/label.json?search=purpose:anxiety&limit=10"
pain_URL = "https://api.fda.gov/drug/label.json?search=purpose:pain&limit=10"
dementia_URL = "https://api.fda.gov/drug/label.json?search=purpose:dementia&limit=100"

### **Top 10 Drugs for Anxiety**

In [ ]:
# Top 10 drugs for anxiety
data = requests.get(anx_URL).json()
anxiety_top10 = pd.DataFrame(data.get("results"))
#anxiety_top10.shape
anxiety_top10

,spl_product_data_elements,spl_unclassified_section,active_ingredient,active_ingredient_table,purpose,purpose_table,indications_and_usage,warnings,ask_doctor,stop_use,...,inactive_ingredient,questions,package_label_principal_display_panel,set_id,id,effective_time,version,openfda,references,other_safety_information
0,"[Calm Mood Garlic, Avena Sativa Flowering Top,...","[Drug Facts, Manufactured & Distributed by For...",[Active ingredients Purpose Allium sativum USD...,"[<table width=""60%""><col width=""40%"" align=""le...",[Active ingredients Purpose Allium sativum USD...,"[<table width=""60%""><col width=""40%"" align=""le...",[Uses for the relief of nervousness and tensio...,[Warnings For oral (sublingual) use. Ask a doc...,[Ask a doctor before use in children under 6 y...,[Stop use and ask doctor if symptoms persist f...,...,"[Inactive ingredients Distilled Water, Grain A...",[Questions or comments? Call us at 1-877-975-3...,[PRINCIPAL DISPLAY PANEL - 10 ml Bottle Carton...,0030a8b4-f70f-415d-bdb6-e2e70720b7a2,4507a3bf-2f07-4bde-9e28-399e121f9162,20210709,2,"{'brand_name': ['Calm Mood'], 'generic_name': ...",NaN,NaN
1,[Serotonin Seotonin SEROTONIN SEROTONIN WATER ...,NaN,[ACTIVE INGREDIENTS: Serotonin 6X.],NaN,[INDICATIONS: For temporary relief of nervousn...,NaN,[INDICATIONS: For temporary relief of nervousn...,"[WARNINGS: If pregnant or breast-feeding, ask ...",NaN,NaN,...,[INACTIVE INGREDIENTS: Demineralized water. 25...,[Distributed By: BioActive Nutritional Melbour...,[BioActive Homeopathic SEROTONIN 6X 1 FL OZ (3...,00cc21e5-39b4-440f-b220-36b2f23cc5f1,b35d4237-6db4-4595-a3c7-fd9e66f5b421,20150804,2,{},NaN,NaN
2,"[FNG Berberis Vulgaris, Cochlearia Armoracia, ...",NaN,"[ACTIVE INGREDIENTS: Berberis vulgaris 3X, Coc...",NaN,[INDICATIONS: For temporary relief of symptoms...,NaN,[INDICATIONS: For temporary relief of symptoms...,[WARNINGS: Keep out of reach of children. In c...,NaN,NaN,...,"[INACTIVE INGREDIENTS: Demineralized water, 25...","[Dist. By: Deseret Biologicals, Inc. 469 Parkl...",[DESBIO NDC 43742-0037-1 HOMEOPATHIC FNG 1 FL ...,00efc759-be02-46b9-b744-bae4590bda81,86318293-b536-4feb-8a9b-0edcc6d5a359,20151021,4,{},NaN,NaN
3,"[CARDIO TONIC Cactus grandiflorus, L-carnitine...",NaN,"[ACTIVE INGREDIENTS: Cactus Grandiflorus 3X, L...",NaN,[INDICATIONS: For temporary relief of nervous ...,NaN,[INDICATIONS: For temporary relief of nervous ...,"[WARNINGS: If pregnant or breast-feeding, ask ...",NaN,NaN,...,"[INACTIVE INGREDIENTS: Demineralized water, 20...","[QUESTIONS: Dist. by Energique, Inc. 201 Apple...",[PACKAGE LABEL DISPLAY: ENERGIQUE SINCE 1987 H...,00fed60e-d10e-403f-bdd2-d036c826b119,af4d9aa6-a820-40bc-9ca7-d9a1674c7cc3,20170316,2,{},NaN,NaN
4,"[Septaforce Baptisia tinctoria, Echinacea, Ars...",NaN,"[ACTIVE INGREDIENTS: Baptisia tinctoria 3X, Ec...",NaN,[INDICATIONS: For temporary relief of constric...,NaN,[INDICATIONS: For temporary relief of constric...,"[WARNINGS: If pregnant or breast-feeding, ask ...",NaN,NaN,...,"[INACTIVE INGREDIENTS: Demineralized water, 25...",[Distributed by: BIOACTIVE NUTRITIONAL Melbour...,[PACKAGE LABEL DISPLAY: BioActive Homeopathic ...,01f88137-00bf-4949-a693-15fb2c3e82fa,03caab76-b70b-4909-a871-fead202eeaa1,20151231,3,{},NaN,NaN
5,"[ANXIETY/STRESS ASA FOETIDA,IGNATIA AMARA,VALE...",NaN,[ACTIVE INGREDIENTS HPUS Asa foetida 3X Ignati...,NaN,[PURPOSE anxiety irritability fatique moodiness],NaN,[USES This is a natural Homeopathic remedy tha...,[WARNINGS],"[If symptoms persist or worsen, discontinue us...",NaN,...,"[INACTIVE INGREDIENTS Lactose, sucrose]",NaN,[CARTON image description],02c97a4f-a808-496d-810b-54312c50bedd,8d5300bb-ba7c-4c50-ba98-b23bd08b4dab,20130523,1,{},[Manufactured according to the Homeopathic Pha...,[OTHER INFORMATION Do not use if cap seal is b...
6,[Vitality Drops hGH Somotropin SOMATROPIN SOMA...,NaN,"[​Ingredients: ​hGH (Somotropin) 6C, 100C, 200...",NaN,[CLINICAL PROVEN USES: • Reduce Fat • Increase...,NaN,[CLINICAL PROVEN USES: • Reduce Fat • Increase..

### **Top 10 Drugs for Pain**

In [ ]:
# Top 10 drugs for pain
data = requests.get(pain_URL).json()
pain_top10 = pd.DataFrame(data.get("results"))
#pain_top10.shape
pain_top10

,effective_time,purpose,keep_out_of_reach_of_children,when_using,questions,pregnancy_or_breast_feeding,storage_and_handling,indications_and_usage,set_id,id,...,openfda,version,dosage_and_administration,stop_use,do_not_use,package_label_principal_display_panel,information_for_patients,overdosage,spl_unclassified_section,other_safety_information
0,20171112,[Purpose Pain reliever/fever reducer],[Keep out of reach of children. In case of ove...,[When using this product • take with food or m...,[Questions or comments? 1-888-309-9030],"[If pregnant or breast-feeding, ask a health p...",[Other information • each capsule contains: so...,[Uses • temporarily relieves minor aches and p...,00051a01-1c2e-4abb-b04c-548401f822c6,bdafad45-6825-4cf2-bb9a-a2ff6ad2e581,...,{},4,[Directions • do not take more than directed •...,[Stop use and ask a doctor if • you experience...,[Do not use • if you have ever had an allergic...,[Package/Label Principal Display Panel Naproxe...,NaN,NaN,NaN,NaN
1,20230328,[Purpose Pain reliever/fever reducer],[Keep out of reach of children. In case of ove...,NaN,[Questions or comments? 1-800-426-9391],"[If pregnant or breast-feeding, ask a health p...",[Other information store at 25°C (77°F); excur...,[Uses temporarily relieves minor aches and pai...,0005aafa-08f8-4990-a54d-cc097195708f,0720b0ce-2979-4984-a259-52092cfaebdc,...,"{'application_number': ['part343'], 'brand_nam...",16,[Directions do not take more than directed adu...,[Stop use and ask a doctor if new symptoms occ...,[Do not use if you are allergic to acetaminoph...,[Principal Display Panel Best Choice 44-519 Co...,NaN,NaN,NaN,NaN
2,20141207,"[Reduces occasional pain, pressure and inflamm...",[Keep this and all medication out of reach of ...,NaN,NaN,"[If pregnant or breastfeeding, ask a health pr...",[Tamper resistant for your protection. Use onl...,[Uses: Homeopathic medicine for temporary reli...,000a466f-ae34-4a25-84ec-e97d15a9f3b7,89aa506d-64d5-4c44-8ef1-0f3a2d3ca183,...,{},1,[Directions: For all ages: Mix 0.50 mL in 1/4 ...,NaN,NaN,[Bottle Label],[The letters HPUS indicate that the components...,"[In case of overdose, get medical help or cont...",NaN,NaN
3,20220407,[Purpose Pain reliever/fever reducer],[Keep out of reach of children. Overdose warni...,NaN,[Questions or comments? 1-800-719-9260],"[If pregnant or breast-feeding, ask a health p...",[Other information • store at 20-25 ° C (68-77...,[Uses • temporarily relieves minor aches and p...,00146b91-008d-4b16-95b0-6077f98821be,8c1bf22a-2d18-4575-8a3d-1f14521d9052,...,"{'application_number': ['part343'], 'brand_nam...",2,[Directions • do not take more than directed (...,[Stop use and ask a doctor if • pain gets wors...,[Do not use • with any other drug containing a...,[Package/Label Principal Display Panel Extra S...,NaN,NaN,NaN,NaN
4,20190415,[Purposes Acetaminophen..........................,[Keep this and all drugs out of the reach of c...,[When using this product • avoid alcoholic dri...,NaN,[If pregnant or breast-feeding ask a health pr...,NaN,[Uses temporarily relieves these common cold/f...,001c9dc0-81cb-454a-a676-a5242e71d386,06e429a5-0429-4899-a0d5-37fa340ccd4e,...,{},4,[Directions • take only as recommended (see ov...,[Stop use and ask a doctor if • redness or swe...,[Do Not Use • if you are on a sodium-restricte...,[Principal Display Panel Premier Value Day-tim...,NaN,NaN,[Other Information • sodium content per tables...,NaN
5,20181214,[Purpose Pain reliever/fever reducer],[Keep out of reach of children Keep out of rea...,[When using this product take with food or mil...,[Questions or Comments? Call1-888-952-0050 Mon...,"[If pregnant or breast-feeding, If pregnant or...",NaN,[Uses temporarily relieves minor aches and pai...,002052df-83ac-4efd-926c-b60a8f64cec4,3441a647-8818-4c13-9ba8-6a012e46b508,...,"{'application_number': ['ANDA090545'], 'brand_...",1,[Directions do not take more than directed the...,[Stop use and ask a doctor if you experience a...,[Do not use if you have ever had an a

### **Top 10 Drugs for Dementia**

In [ ]:
# Top 10 drugs for dementia
data = requests.get(dementia_URL).json()
dementia_top10 = pd.DataFrame(data.get("results"))
#dementia_top10.shape
dementia_top10

,effective_time,inactive_ingredient,purpose,keep_out_of_reach_of_children,warnings,spl_product_data_elements,openfda,version,dosage_and_administration,spl_unclassified_section,package_label_principal_display_panel,indications_and_usage,set_id,id,active_ingredient,questions,storage_and_handling
0,20180702,"[Inactive Ingredients Sucrose, Lactose]",[Purpose Helps improve forgetfulness and demen...,[Keep out of reach of children Keep out of rea...,[Warnings If pregnant or breast feeding ask a ...,"[Hardmet Car 40 (Number 90) Argentum Nitricum,...",{},3,[Direction Adult or child: Take three pills da...,[Manufactured by Paramesh Banerji Life Science...,[Principal Display Panel NDC: 69152-0090-1 Har...,[Uses Helps improve forgetfulness and dementia],070c6f25-4e71-3466-e054-00144ff88e88,7005a84a-e7ab-bf7a-e053-2a91aa0a0061,[DRUG FACTS: Active Ingredients Argentum nitri...,NaN,NaN
1,20180627,"[INACTIVE INGREDIENTS: Demineralized Water, 25...",[INDICATIONS: For the temporary relief of loss...,[KEEP OUT OF REACH OF CHILDREN: Keep out of re...,"[WARNINGS: If pregnant or breast-feeding, ask ...","[Neurobalance Pro Adrenocorticotrophin, GABA, ...","{'brand_name': ['Neurobalance Pro'], 'generic_...",2,"[DIRECTIONS: Spray orally once or twice daily,...",NaN,[PACKAGE DISPLAY LABEL: NDC#: 50181-0041-1 The...,[INDICATIONS: For the temporary relief of loss...,70251489-1511-485e-bb95-6eaac218c27b,4f2d0d1d-4fc4-4701-b0fb-9d557dd7f90b,"[ACTIVE INGREDIENTS: Adrenocorticotrophin 6X, ...",[QUESTIONS: MFD FOR: The Wellness Center for R...,NaN
2,20191119,"[INACTIVE INGREDIENTS: Demineralized Water, 25...",[INDICATIONS: For the temporary relief of loss...,[KEEP OUT OF REACH OF CHILDREN: Keep out of re...,"[WARNINGS: If pregnant or breast-feeding, ask ...",[Neurobalance Pro GABA (Gamma-Aminobutyric Aci...,{},1,"[DIRECTIONS: Spray orally once or twice daily,...",NaN,[PACKAGE DISPLAY LABEL: NDC#: 50181-0070-1 The...,[INDICATIONS: For the temporary relief of loss...,a37237ed-6465-4623-974d-4f9e22d1e8e9,5d8e5108-7d03-4953-b79d-87075365df6d,[ACTIVE INGREDIENTS: GABA (Gamma-Aminobutyric ...,[QUESTIONS: MFD FOR: The Wellness Center for R...,NaN
3,20100709,"[INACTIVE INGREDIENTS: Demineralized Water, 25...",[INDICATIONS: For the temporary relief of loss...,NaN,"[WARNINGS: If pregnant or breast-feeding, ask ...","[Neuro Balance Pro Adrenocorticotrophin, Gaba,...",{},1,"[DIRECTIONS: Spray orally once or twice daily,...",NaN,[Theresa Dale's NEURO BALANCE PRO HOMEOPATHIC ...,NaN,b3abed0f-4d8c-42b5-8b28-f7bfb3fcb0eb,93793271-55d2-48b4-a921-83f1759236ce,"[ACTIVE INGREDIENTS: Adrenocorticotrophin 6X, ...","[MFG FOR: Theresa Dale, N. D. 1237 S. Victoria...",NaN
4,20130809,"[INACTIVE INGREDIENTS: Demineralized Water, 25...",[INDICATIONS: For the temporary relief of loss...,[KEEP OUT OF REACH OF CHILDREN: ​Keep out of r...,"[WARNINGS: ​If pregnant or breast-feeding, ​ a...","[NEUROBALANCE PRO Adrenocorticotrophin, Gaba, ...",{},1,"[DIRECTIONS: Spray orally once or twice daily,...",NaN,[PACKAGE DISPLAY LABEL: Theresa Dale's ​NeuroB...,[INDICATIONS: For the temporary relief of loss...,b6637f00-3660-4fc5-acbd-4f1dfb02e80d,87dda463-afe6-4675-9347-af54ca5d4354,"[ACTIVE INGREDIENTS: Adrenocorticotrophin 6X, ...",[QUESTIONS: MFD. FOR: The Wellness Center for ...,"[STORAGE: Store in a cool, dry place.]"


### **Pagination**

In [ ]:
def get_all_psychiatric_drugs():
    base_url = "https://api.fda.gov/drug/label.json"

    # Parameters
    params = {
        'search': 'purpose:psychiatric',
        'limit': 100  # setting limit to 100 currently, can be adjusted if we need to later
    }

    all_psychiatric_drugs = []

    #Because it can be hard to know which keywords to use, we can use this check function to pull only if present.
    try:
        while True:
            response = requests.get(base_url, params=params)
            response.raise_for_status()

            data = response.json()

            # Check if any results were found
            if 'results' in data:
                drugs_data = data['results']

                all_psychiatric_drugs.extend(drugs_data)

                # Pagination
                if 'meta' in data and 'results' in data['meta']:
                    total_results = data['meta']['results']['total']
                    retrieved_results = len(all_psychiatric_drugs)

                    if retrieved_results < total_results:
                        params['skip'] = retrieved_results
                    else:
                        break
                else:
                    break
            else:
                print("No data found.")
                break

        return all_psychiatric_drugs

    except requests.exceptions.RequestException as e:
        print(f"An error occurred: {e}")
        return []

if __name__ == "__main__":
    psychiatric_drugs_data = get_all_psychiatric_drugs()

    if psychiatric_drugs_data:
        psychiatric_df = pd.DataFrame(psychiatric_drugs_data)

        # Space to rename columns later, some column names are difficult to read
        #psychiatric_df.columns = []

    else:
        print("No data found.")


 # Display the top 5 drugs
psychiatric_df.head(5)

,spl_product_data_elements,active_ingredient,indications_and_usage,warnings,keep_out_of_reach_of_children,dosage_and_administration,other_safety_information,inactive_ingredient,questions,package_label_principal_display_panel,purpose,set_id,id,effective_time,version,openfda
0,"[Earth Aur met, Arg nit, Causticum, Cinchona, ...",[​ACTIVE INGREDIENTS (HPUS*) ​: Contains equal...,[​INDICATIONS FOR USE: For the temporary relie...,"[​WARNINGS: • ​If symptoms persist or worsen, ...",[• Keep this and all medication out of reach o...,[​ ​DIRECTIONS: ​Spray 9-12 pumps (approximate...,[​OTHER INFORMATION ​: Tamper resistant for yo...,"[​INACTIVE INGREDIENTS ​: Citric acid, Potassi...",[​www.NETmindbody.com Made in the USA],[​NET Remedies #1 Earth Homeopathic Preparatio...,[For the temporary relief of the neuro-emotion...,06bfa185-71a2-4297-9a02-a9951a004bc5,b9491337-22e4-60be-e053-2995a90a2ca0,20210119,2,"{'brand_name': ['Earth'], 'generic_name': ['AU..."
1,"[Water Aconitum nap, Arsenicum alb, Borax, Cal...",[​ACTIVE INGREDIENTS (HPUS*) ​: Contains equal...,[​INDICATIONS FOR USE: ​For the temporary reli...,[​WARNINGS: • ​If symptoms persist or worsen ​...,[• Keep this and all medication out of reach o...,[​DIRECTIONS: ​Spray 9-12 pumps (approximately...,[​OTHER INFORMATION: ​Tamper resistant for you...,"[​INACTIVE INGREDIENTS: ​Citric acid, Potassiu...",[​www.NETmindbody.com ​Made in the USA.],[​NET Remedies #3 Water Homeopathic preparatio...,[For the temporary relief of the neuro-emotion...,3dc6fdf6-781c-4dec-9aab-a2237fc60c53,b9492369-ac4d-7b94-e053-2995a90ad837,20210119,2,"{'brand_name': ['Water'], 'generic_name': ['AC..."
2,"[Fire Apil mel, Baptisia, Baryta carb, Hamamel...",[​ACTIVE INGREDIENTS (HPUS*) ​: Contains equal...,[​INDICATIONS FOR USE: ​For temporary relief o...,[​WARNINGS : ​• ​If symptoms persist or worsen...,[• Keep this and all medication out of reach o...,[​DIRECTIONS: ​Spray 9-12 pumps (approximately...,[​OTHER INFORMATION ​: Tamper resistant for yo...,"[​INACTIVE INGREDIENTS ​: Citric acid, Potassi...",[​www.NETmindbody.com Made in the USA],[​NET Remedies #5 Fire Homeopathic Preparation...,[For temporary relief of the neuro-emotional (...,684d7ff2-b241-4c4e-9a16-14c9adfc260f,b9492b5a-907e-4f3c-e053-2a95a90a6650,20210119,2,"{'brand_name': ['Fire'], 'generic_name': ['API..."
3,[Xpect Cough Relief Dextromethorphan HBr Guaif...,[Dextromethorphan HBr 10mg Guaifenesein 100mg],[Uses Temporarily: • Relieves coughing due to ...,[Warnings Do not use this product for persiste...,[Keep out of reach of children. In case of ove...,[Directions • Adults and children 12 years and...,"[• Do not use if dispenser is torn, cut or ope...","[Citric acid, FD&C red #40, flavor, glycerin, ...",NaN,[Cough Relief label-01],[Purpose Active ingredients (in each tablet) D...,9d36cf74-829a-477e-95f9-8ccd184c567e,e3e35864-0db2-4d36-9daf-04f263792bd8,20221231,4,{}
4,[Xpect Sinus Relief Acetaminophen/Phenylephrin...,[Acetaminophen 325mg Phenylephrine HCl 5mg],[Uses Temporarily relieves: • Sinus pain • Hea...,[Warnings Liver Warning: This product contains...,[Keep out of reach of children Overdose warnin...,[Directions • Adults and children 12 years of ...,[Other Information Do not use if packet is tor...,"[FD&C Blue #1, Hypromellose, maltodextrin, mic...",NaN,[label-01],[Purpose Active ingredients (in each tablet) A...,c4e0962d-b539-47bb-bfdc-11ad7b079cf1,851faaa4-db63-476e-82c2-39f0a873051e,20221231,4,{}


### **Would like to wrap so we dont have to pull each condition**

This is currently a work in progress - need to update for multi word conditions

In [ ]:
def get_drugs_by_purpose(purposes):
    base_url = "https://api.fda.gov/drug/label.json"
    all_drugs_data = []

    for purpose in purposes:
        # Define parameters for the API request
        params = {
            'search': f'purpose:{purpose}',
            'limit': 100  # had set to 100, was taking ~30 min
        }

        try:
            while True:
                response = requests.get(base_url, params=params)
                response.raise_for_status()  # Raise an exception for HTTP errors

                data = response.json()

                # Check if any results were found
                if 'results' in data:
                    drugs_data = data['results']

                    all_drugs_data.extend(drugs_data)

                    # Pagination
                    if 'meta' in data and 'results' in data['meta']:
                        total_results = data['meta']['results']['total']
                        retrieved_results = len(all_drugs_data)

                        if retrieved_results < total_results:
                            params['skip'] = retrieved_results
                        else:
                            break
                    else:
                        break
                else:
                    print(f"No data found for purpose: {purpose}")
                    break

        except requests.exceptions.RequestException as e:
            print(f"An error occurred: {e}")

    return all_drugs_data

In [ ]:
if __name__ == "__main__":
    purposes = ['anxiety', 'depression', 'dementia'] #'high blood pressure'
    all_drugs_data = get_drugs_by_purpose(purposes)

    if all_drugs_data:
        # Create a DataFrame from the retrieved data
        drugs_df = pd.DataFrame(all_drugs_data)

        # Space to rename columns later, some column names are difficult to read
        #psychiatric_df.columns = []

    else:
        print("No data found for the specified purposes.")


In [ ]:
drugs_df.head(5)

,spl_product_data_elements,spl_unclassified_section,active_ingredient,active_ingredient_table,purpose,purpose_table,indications_and_usage,warnings,ask_doctor,stop_use,...,description,when_using,dosage_forms_and_strengths,description_table,inactive_ingredient_table,package_label_principal_display_panel_table,pregnancy,general_precautions,information_for_owners_or_caregivers,recent_major_changes
0,"[Calm Mood Garlic, Avena Sativa Flowering Top,...","[Drug Facts, Manufactured & Distributed by For...",[Active ingredients Purpose Allium sativum USD...,"[<table width=""60%""><col width=""40%"" align=""le...",[Active ingredients Purpose Allium sativum USD...,"[<table width=""60%""><col width=""40%"" align=""le...",[Uses for the relief of nervousness and tensio...,[Warnings For oral (sublingual) use. Ask a doc...,[Ask a doctor before use in children under 6 y...,[Stop use and ask doctor if symptoms persist f...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,[Serotonin Seotonin SEROTONIN SEROTONIN WATER ...,NaN,[ACTIVE INGREDIENTS: Serotonin 6X.],NaN,[INDICATIONS: For temporary relief of nervousn...,NaN,[INDICATIONS: For temporary relief of nervousn...,"[WARNINGS: If pregnant or breast-feeding, ask ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[FNG Berberis Vulgaris, Cochlearia Armoracia, ...",NaN,"[ACTIVE INGREDIENTS: Berberis vulgaris 3X, Coc...",NaN,[INDICATIONS: For temporary relief of symptoms...,NaN,[INDICATIONS: For temporary relief of symptoms...,[WARNINGS: Keep out of reach of children. In c...,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[CARDIO TONIC Cactus grandiflorus, L-carnitine...",NaN,"[ACTIVE INGREDIENTS: Cactus Grandiflorus 3X, L...",NaN,[INDICATIONS: For temporary relief of nervous ...,NaN,[INDICATIONS: For temporary relief of nervous ...,"[WARNINGS: If pregnant or breast-feeding, ask ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Septaforce Baptisia tinctoria, Echinacea, Ars...",NaN,"[ACTIVE INGREDIENTS: Baptisia tinctoria 3X, Ec...",NaN,[INDICATIONS: For temporary relief of constric...,NaN,[INDICATIONS: For temporary relief of constric...,"[WARNINGS: If pregnant or breast-feeding, ask ...",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#Using broader topics based on professor comments, running w/ limit 100 overnight. Limit 10 had 14,945 rows. Hoping for more for a complete dataset.

if __name__ == "__main__":
    purposes = ['psychiatric', 'pain', 'cholesterol', 'hypertension', 'diabetes', 'infection', 'cancer', 'asthma', 'allergies']
    new_all_drugs_data = get_drugs_by_purpose(purposes)

    if new_all_drugs_data:
        # Create a DataFrame from the retrieved data
        new_drugs_df = pd.DataFrame(new_all_drugs_data)

        # Space to rename columns later, some column names are difficult to read
        #psychiatric_df.columns = []

    else:
        print("No data found for the specified purposes.")

# Save for overnight storage
file_path = "/home/alldrugs.txt"
new_drugs_df.to_csv(file_path, sep='\t', index=False)

# Check if the file was saved
import os
if os.path.exists(file_path):
    print(f"DataFrame saved as {file_path}")
else:
    print("Error: DataFrame not saved")

DataFrame saved as /home/alldrugs.txt


In [ ]:
#new_drugs_df.head(5)
new_drugs_df.shape

(15358, 101)

***

## **Data Preprocessing**

### **Preprocessing Functions**
* Lowercase text: Convert all text to lowercase, so that capitalization does not affect processing.

* Remove punctuation: Eliminate any punctuation marks from the text.

* Tokenize: Split the text into individual words or tokens.

* Remove stopwords: Filter out common stopwords from the text.

* Join tokens: Reconstruct the processed tokens into a single string or text.

In [ ]:
# Create a set of stopwords
sw = set(stopwords.words('english'))

# Create a set of punctuation marks
punctuation = set(string.punctuation)  # Speeds up comparison
tw_punct = punctuation - {"#"}

# Remove stopwords
def remove_stop(tokens):
    return [w for w in tokens if w.lower() not in sw]

# Remove punctuation
def remove_punctuation(text, punct_set=tw_punct):
    return "".join([ch for ch in text if ch not in punct_set])
a
# Tokenize
def tokenize(text):
    return text.split()

# Convert to lowercase
def lowercase(text):
    return text.lower()

# Joins tokens (concatenation)
def join_tokens(tokens):
    text = " ".join(tokens)
    return text

# Applies pipeline to text
def prepare(text, pipeline):
    tokens = str(text)
    for transform in pipeline:
        tokens = transform(tokens)
    return tokens

cleaning_pipeline = [lowercase, remove_punctuation, tokenize, remove_stop, join_tokens]

### **Cleaning Anxiety Dataframe**

In [ ]:
# Clean anxiety dataframe

# Apply the cleaning pipeline to all text columns at once
text_columns = anxiety_top10.columns
cleaned_columns = [col + '_cleaned' for col in text_columns]

anxiety_top10[cleaned_columns] = anxiety_top10[text_columns].apply(lambda col: col.apply(lambda x: prepare(x, cleaning_pipeline)))

# Drop the original columns
anxiety_top10 = anxiety_top10.drop(columns = text_columns)

# Display the cleaned dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
anxiety_top10.head()

### **Cleaning Pain Dataframe**

In [ ]:
# Clean pain dataframe

# Apply the cleaning pipeline to all text columns at once
text_columns = pain_top10.columns
cleaned_columns = [col + '_cleaned' for col in text_columns]

pain_top10[cleaned_columns] = pain_top10[text_columns].apply(lambda col: col.apply(lambda x: prepare(x, cleaning_pipeline)))

# Drop the original columns
pain_top10 = pain_top10.drop(columns = text_columns)

# Define a function to count tokens in a given list of tokens
def count_tokens(tokens):
    return len(tokens)

# Apply the count_tokens function to the 'tokens' column and create a new column 'num_tokens'
pain_top10['num_tokens'] = dementia_top10['tokens'].apply(count_tokens)

# Display the cleaned dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pain_top10.head()

### **Cleaning Dementia Dataframe**

In [ ]:
# Clean pain dataframe

# Apply the cleaning pipeline to all text columns at once
text_columns = pain_top10.columns
cleaned_columns = [col + '_cleaned' for col in text_columns]

pain_top10[cleaned_columns] = pain_top10[text_columns].apply(lambda col: col.apply(lambda x: prepare(x, cleaning_pipeline)))

# Drop the original columns
pain_top10 = pain_top10.drop(columns = text_columns)

# Display the cleaned dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
pain_top10.head()

### **Cleaning New Drugs Dataframe**

In [ ]:
# Clean new drugs dataframe

# Apply the cleaning pipeline to all text columns at once
text_columns = new_drugs_df.columns
cleaned_columns = [col + '_cleaned' for col in text_columns]

# Apply the cleaning pipeline to all text columns at once using pd.concat
cleaned_data = pd.concat([new_drugs_df[col].apply(lambda x: prepare(x, cleaning_pipeline)) for col in text_columns], axis=1)
cleaned_data.columns = cleaned_columns

# Combine cleaned data with the original DataFrame
new_drugs_df = pd.concat([new_drugs_df, cleaned_data], axis=1)

# Drop the original columns
new_drugs_df = new_drugs_df.drop(columns = text_columns)

# Display the cleaned dataframe
pd.set_option('display.max_columns', None)
pd.set_option('display.expand_frame_repr', False)
new_drugs_df.head()

***

## **Exploratory Data Analysis**

### **Descriptive Statistics**

In [ ]:
def descriptive_stats(tokens, num_tokens = 5, verbose=True):
    num_tokens = len(tokens)
    num_unique_tokens = len(set(tokens))
    lexical_diversity = num_unique_tokens / num_tokens
    num_characters = sum(len(token) for token in tokens)

    if verbose:
        print(f"There are {num_tokens} tokens in the data.")
        print(f"There are {num_unique_tokens} unique tokens in the data.")
        print(f"There are {num_characters} characters in the data.")
        print(f"The lexical diversity is {lexical_diversity:.3f} in the data.")
        print(Counter(tokens).most_common(num_tokens))  # Display the most common tokens

    return [num_tokens, num_unique_tokens, lexical_diversity, num_characters]

#### **Descriptive Statistics: Anxiety**

In [ ]:
anxiety_top10.info()

In [ ]:
anxiety_top10.describe()

In [ ]:
anxiety_top10.isnull().sum()

In [ ]:
# Apply the descriptive_stats function to the 'tokens' column
anxiety_top10['descriptive_stats'] = anxiety_top10['tokens'].apply(descriptive_stats)
anxiety_top10['tokens_description'] = anxiety_top10['tokens'].apply(lambda x: descriptive_stats(x, num_tokens = 5, verbose = True))

# Print the resulting dataframe with the descriptive statistics
anxiety_top10[['tokens', 'tokens_description']]

#### **Descriptive Statistics: Pain**

In [ ]:
pain_top10.info()

In [ ]:
pain_top10.describe()

In [ ]:
pain_top10.isnull().sum()

In [ ]:
dementia_top10.info()

In [ ]:
dementia_top10.describe()

In [ ]:
dementia_top10.isnull().sum()

In [ ]:
# Tokenize the selected text columns and store them in separate columns
for col in text_columns:
    col_name = col + '_tokens'
    dementia_top10[col_name] = dementia_top10[col].apply(word_tokenize)

# Combine all token columns into a single 'tokens' column
dementia_top10['tokens'] = dementia_top10[text_columns].apply(lambda x: ' '.join(x), axis = 1)

# Count the number of tokens and store it in the 'num_tokens' column
dementia_top10['num_tokens'] = dementia_top10['tokens'].apply(lambda text: len(word_tokenize(text)))

# Print the updated DataFrame with 'tokens' and 'num_tokens' columns
dementia_top10[['tokens', 'num_tokens']].head()

In [ ]:
# Apply the descriptive_stats and tokens_description functions to the 'tokens' column
dementia_top10['descriptive_stats'] = dementia_top10['tokens'].apply(descriptive_stats)
dementia_top10['tokens_description'] = dementia_top10['tokens'].apply(lambda x: descriptive_stats(x, num_tokens = 5, verbose = True))

# Print the resulting dataframe with the descriptive statistics
dementia_top10[['tokens', 'tokens_description']]

In [ ]:
# Define the column to analyze
dementia_target_column = "active_ingredient_cleaned"

# Extract all tokens from the specified column
all_tokens = [token for tokens in dementia_top10[dementia_target_column].apply(word_tokenize) for token in tokens]

# Calculate descriptive statistics for the selected column
descriptive_stats(all_tokens)

### **Bar Plots**

#### **Bar Plots: Anxiety**

In [ ]:
# Visualize all non-numeric columns (including categorical)
non_numeric_columns = anxiety_top10.select_dtypes(exclude = ['int64', 'float64'])

for col in non_numeric_columns.columns:
    if col in numeric_columns.columns:
        continue  # Skip columns that are also numeric
    sns.countplot(data = anxiety_top10, x = col)
    plt.title(f"Bar Plot of {col}")
    plt.xticks(rotation = 45)
    plt.show()

#### **Bar Plots: Pain**

In [ ]:
# Visualize all non-numeric columns (including categorical)
non_numeric_columns = pain_top10.select_dtypes(exclude = ['int64', 'float64'])

for col in non_numeric_columns.columns:
    if col in numeric_columns.columns:
        continue  # Skip columns that are also numeric
    sns.countplot(data = pain_top10, x = col)
    plt.title(f"Bar Plot of {col}")
    plt.xticks(rotation = 45)
    plt.show()

#### **Bar Plots: Dementia**

In [ ]:
# Visualize all non-numeric columns (including categorical)
non_numeric_columns = dementia_top10.select_dtypes(exclude = ['int64', 'float64'])

for col in non_numeric_columns.columns:
    if col in numeric_columns.columns:
        continue  # Skip columns that are also numeric
    sns.countplot(data = dementia_top10, x = col)
    plt.title(f"Bar Plot of {col}")
    plt.xticks(rotation = 45)
    plt.show()